In [40]:
import pandas as pd
import numpy as np
from glob import glob
from re import match, sub
from missingno import matrix as mmatrix
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from math import ceil
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from statsmodels.formula.api import ols
from os import listdir, getenv
from sys import stdout
from collections import defaultdict
from tqdm import tqdm
from itertools import product
from json import dumps
from copy import copy

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, LSTM, Bidirectional
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, MAPE, MeanAbsolutePercentageError, MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, History, ModelCheckpoint
from tensorflow.keras.regularizers import L1


In [41]:
def scheduler1(epoch, lr): # Winner
    return 0.0001 if epoch % 2 == 1 else 0.001
    
def scheduler2(epoch, lr):
    return 0.0001 if ((epoch % 2 == 1) or (epoch > epochs/2)) else 0.001
    
def scheduler3(epoch, lr):
    return 0.0001

def scheduler4(epoch, lr):
    return 0.0001 if ((epoch % 2 == 1) or (epoch > epochs-100)) else 0.001

class ProgressBar(Callback):

    # def __init__(self, schedule):
    #     super().__init__()

    def on_train_begin(self, logs=None):
        self.epochs = self.__dict__['params']['epochs']
        self.steps = self.__dict__['params']['steps'] 
        self.total_steps = self.steps * self.epochs
        self.progress_bar = tqdm(
            desc='Training', total=self.total_steps, unit=' step', smoothing=0,
            file=stdout, colour='green', position=0, ncols=100, unit_scale=1/self.steps,
            bar_format='{l_bar}{bar}| {n:.0f}/{total:.0f} epochs completed [{elapsed}<{remaining}, {rate_fmt}]',
        )
        # if 'MLP' in self.model.name:
        #     self.epoch_update = self.steps
        #     self.batch_update = 0
        # else:
        #     self.epoch_update = 0
        #     self.batch_update = 1

    # def on_epoch_end(self, batch, logs=None):
    #     self.progress_bar.update(self.epoch_update)

    def on_batch_end(self, batch, logs=None):
        self.progress_bar.update(1)

    def on_train_end(self, logs=None):
        self.progress_bar.close()

class SaveTrainTime(Callback):

    def on_train_begin(self, logs=None):
        self.start_time = datetime.now()

    def on_train_end(self, logs=None):
        ttimes = pd.read_csv('models/train_times.csv')
        ttimes.train_time = ttimes.train_time.astype('timedelta64[ns]')
        params = self.__dict__['params']
        ttimes = pd.concat([
            ttimes,
            pd.DataFrame([[
                self.model.name, 
                self.start_time, 
                datetime.now() - self.start_time, 
                params['epochs'], 
                params['steps'],
            ]], columns=ttimes.columns) # model,start_time,train_time,epochs,steps
        ])
        
        ttimes.to_csv('models/train_times.csv', index=False)

def get_MLP(X_train, n_hidden=1, univariate=False, model_prefix=''):
    """Build and train a multilayer perception. Return the uncompiled model.

    Args:
        X_train (tf.Tensor): Input training data.
        n_hidden (int): Number of hidden layer in neural network.

    Returns:
        model (keras.src.models.functional.Functional): The uncompiled model. !!!!
    """
    tf.keras.utils.set_random_seed(1)
    
    main_input = Input(shape=tuple(X_train[0].shape), name="input")
    previous_layer = main_input
    for i in range(n_hidden):
        if i % 2 == 1:
            previous_layer = Dense(32, name=f"linear_{i+1}", activation='linear')(previous_layer)
        else:
            previous_layer = Dense(32, name=f"relu_{i+1}", activation='relu')(previous_layer)
    
    main_output = Dense(2, name=f"output")(previous_layer)
    
    model_prefix = model_prefix + '_' if model_prefix else model_prefix
    name = f"{model_prefix}MLP_{'shallow' if n_hidden <= 1 else 'deep'}_{'univ' if univariate else 'multiv'}"
    
    return {
        'model': Model(inputs=main_input, outputs=main_output, name=name),
        'univariate': univariate,
        'X_train': X_train,
    }

def get_LSTM(X_train, n_hidden=1, univariate=False, model_prefix=''):
    """Build a bidirectional LSTM. Return the uncompiled model.

    Args:
        X_train (tf.Tensor): Input training data.
        n_hidden (int): Number of hidden layer in neural network.

    Returns:
        model (keras.src.models.functional.Functional): The uncompiled model.
    """
    tf.keras.utils.set_random_seed(1)
    
    main_input = Input(shape=(X_train[0].shape[0],1), name="input")
    previous_layer = main_input
    for i in range(n_hidden-1):
        previous_layer = Bidirectional(
            LSTM(2**(4+n_hidden-i), return_sequences=True), name=f"BD_{i+1}"
        )(previous_layer)
    previous_layer = Bidirectional(LSTM(2**5), name=f"BD_{n_hidden}")(previous_layer)
    
    main_output = Dense(2, name=f"output")(previous_layer)

    model_prefix = model_prefix + '_' if model_prefix else model_prefix
    name = f"{model_prefix}LSTM_{'shallow' if n_hidden <= 1 else 'deep'}_{'univ' if univariate else 'multiv'}"
    
    return {
        'X_train': X_train,
        'model': Model(inputs=main_input, outputs=main_output, name=name),
        'univariate': univariate,
    }

def _compile_model(y_train, X_val, y_val, X_train, model, univariate=False, epochs=2000, opt_params={}):
    """Train an uncompiled Tensorflow model. Return the trained model and its training history.

    Args:
        model (keras.src.models.functional.Functional): The uncompiled model.
        X_train (tf.Tensor): Input training data.
        y_train (tf.Tensor): Target training data.
        X_val (tf.Tensor): Input validation data.
        y_val (tf.Tensor): Target validation data.
        lr_scheduler (function): Function defining the learning rate schedule for fitting the model.
        weight_decay (float): Weight decay hyperparameter value.
        epochs (int): Number of training epochs.
        univariate (bool): True if model is univariate and False if multivariate.
        model_type (str): Label of model family.
        verbose (bool): Verbosity mode when fitting the model.

    Returns:
        model (keras.src.models.functional.Functional): The trained model.
        history (pd.DataFrame): The record of training and validation loss and metric values at successive epochs.
    """
    
    model.compile(
        optimizer=Adam(**opt_params), 
        loss=MeanAbsolutePercentageError(), 
        metrics=[MSE]
    )
    return model

lstm_opt_params = {'weight_decay': 1e-05, 'beta_1': 0.8}

def train_model(y_train, X_val, y_val, X_train, model, univariate=False, epochs=2000, opt_params={}, fit_params={}):
    """Train an uncompiled Tensorflow model. Return the trained model and its training history.

    Args:
        model (keras.src.models.functional.Functional): The uncompiled model.
        X_train (tf.Tensor): Input training data.
        y_train (tf.Tensor): Target training data.
        X_val (tf.Tensor): Input validation data.
        y_val (tf.Tensor): Target validation data.
        lr_scheduler (function): Function defining the learning rate schedule for fitting the model.
        weight_decay (float): Weight decay hyperparameter value.
        epochs (int): Number of training epochs.
        univariate (bool): True if model is univariate and False if multivariate.
        verbose (bool): Verbosity mode when fitting the model.

    Returns:
        model (keras.src.models.functional.Functional): The trained model.
        history (pd.DataFrame): The record of training and validation loss and metric values at successive epochs.
    """
    if 'MLP' in model.name:
        opt_params = opt_params if opt_params else {'weight_decay': 1e-05, 'beta_1': 0.95}
        fit_params = fit_params if fit_params else {'batch_size': 2**8}
        model_checkpoints = ModelCheckpoint(f"models/{model.name}.keras", save_best_only=True)
    elif 'LSTM' in model.name:
        opt_params = opt_params if opt_params else lstm_opt_params
        fit_params = fit_params if fit_params else {'batch_size': 2**8}
        model_checkpoints = ModelCheckpoint(f"models/{model.name}.weights.h5", save_best_only=True, save_weights_only=True)

    model = _compile_model(y_train, X_val, y_val, X_train, model, univariate, epochs=epochs, opt_params=opt_params)
    
    history = model.fit(
        X_train, y_train, verbose=0, shuffle=False, epochs=epochs,
            validation_data=(X_val, y_val), callbacks=[
            model_checkpoints,
            LearningRateScheduler(scheduler1),
            ProgressBar(),
            SaveTrainTime(),
        ], **fit_params
    )
    history = pd.DataFrame(history.history)
    history.to_csv(f'models/{model.name} history.csv', index=False, lineterminator='\n')
    
    return model, history

def flatten_params(param_grid):
    """Take a paramater grid and return a flattened list of each possible combination of the parameters."""
    return [
        dict(zip(param_grid.keys(), e)) 
        for e in product(*param_grid.values())
    ]

def GridSearchCV(estimator, param_grid, epochs):
    """Take a callable model generator and a parameter grid and return the best possible set of parameters given the specified epochs.

    Args:
        estimator (function): A function that returns an uncompiled model.
        param_grid (dict): A dictionary of lists of hyperparameters specifiying the hyperparameter space to be searched.

    Returns:
        dict: The hyperparameters used to train the best model.
    """
    est_params = {}
    opt_params = {}
    params = {}
    for k, v in param_grid.items():
        if (est_lab := 'estimator__') in k:
            est_params.update({k.replace(est_lab, ''): v})
        elif (opt_lab := 'optimizer__') in k:
            opt_params.update({k.replace(opt_lab, ''): v})
        else:
            params.update({k: v})
            
    param_grid_list = flatten_params({
        'est_params': flatten_params(est_params), 
        'opt_params': flatten_params(opt_params), 
        'params': flatten_params(params),
    })
    n_iter = len(param_grid_list)
    results = defaultdict(list)
    
    for i in tqdm(range(n_iter), desc='Searching hyperparameter space', file=stdout, colour='green'):
        param_grid = param_grid_list[i]
        for j in range(len(X_train_sets)):
            X_train, y_train, X_val, y_val = X_train_sets[j], y_train_sets[j], X_val_sets[j], y_val_sets[j]

            est_params = param_grid['est_params']
            opt_params = param_grid['opt_params']
            params = param_grid['params']

            model = estimator(X_train, **est_params)
            
            model.compile(
                optimizer=Adam(**opt_params), 
                loss=MeanAbsolutePercentageError(), 
            )
            history = model.fit(
                X_train, y_train, verbose=0, shuffle=False, epochs=epochs,
                validation_data=(X_val, y_val), callbacks=[
                    LearningRateScheduler(scheduler1)
                ], **params
            )
            results[i].append(model.evaluate(X_val, y_val, verbose=0))

    return param_grid_list[pd.Series(results).apply(np.mean).argmin()]

def load_lstm(name):
    if 'univ' in name:
        _X_train, _y_train, _X_test, _y_test = X_train_uv, y_train_full, X_test_uv, y_test
        univariate = True
    else:
        _X_train, _y_train, _X_test, _y_test = X_train_full, y_train_full, X_test, y_test
        univariate = False
    if 'deep' in name:
        n_hidden = 2
    else:
        n_hidden = 1
    
    prefix = name.split('_LSTM')[0]
    model = _compile_model(
        _y_train, _X_test, _y_test, **get_LSTM(
            _X_train, n_hidden, univariate=univariate, 
            model_prefix=prefix
        ),
    )
    
    model.load_weights(f"models/{name}.weights.h5")

    for k, v in lstm_opt_params.items():
        exec(f'model.optimizer.{k} = {v}')

    return model

def extend_training(model):
    model = copy(model)
    model.name = f"Extended_{model.name}"
    return model

def get_latest_models(name=None):
    all_models = pd.DataFrame([e for e in glob('models/*') if match(r'.*\.(?:keras|h5)$', e)], columns=['file'])
    all_models['model'] = all_models.file.str.extract(r'models/.*Final_(.*?)\..*$')
    all_models['model_name'] = all_models.file.str.extract(r'models/(.*Final_.*?)\..*$')
    all_models['extensions'] = all_models.file.apply(lambda e: e.count('Extended'))
    all_models['latest'] = all_models.groupby('model').extensions.transform('max')
    latest = all_models.loc[
        all_models.extensions == all_models.latest, 
        ['model', 'model_name', 'file']
    ]
    
    if name:
        latest = latest.loc[latest.model == name, 'model_name'].squeeze()
    
    return latest

# _ = train_model(
#     y_train_full, X_test_uv, y_test, 
#     epochs=1, **get_MLP(X_train_uv, 1, univariate=True, model_prefix='test'),
# )


Training:   0%|                    | 0/42 epochs completed [40:11:05<22861:11:47, 1962975.56s/ step]


In [42]:
df = pd.read_csv('data/modelling_data.csv', parse_dates=['DATETIME'], date_format='%Y-%m-%d %H:%M:%S')

y_cols = ['h1_TOTALDEMAND', 'h24_TOTALDEMAND']

df = df.set_index('DATETIME').dropna()

X_df, y_df = df.drop(columns=y_cols), df[y_cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)

X_y = [X_train_full, X_test, y_train_full, y_test] = train_test_split(
    X_scaled, y_df, test_size=0.2, shuffle=False
)
X_y = [tf.convert_to_tensor(d) for d in X_y]
[X_train_full, X_test, y_train_full, y_test] = X_y

uv_cols = np.array([bool(match(r'TOTALDEMAND|TM\d+', e)) for e in X_df.columns])
X_train_uv = tf.convert_to_tensor(X_train_full.numpy()[:,uv_cols])
X_test_uv = tf.convert_to_tensor(X_test.numpy()[:,uv_cols])

train_val_I = TimeSeriesSplit(n_splits=5).split(X_train_full)
X_train_sets, y_train_sets, X_val_sets, y_val_sets = [], [], [], []

for train, val in train_val_I:
    print(train.shape, val.shape)
    val_start, val_end = val.min(), val.max()+1
    X_train_sets.append(X_train_full[:val_start])
    y_train_sets.append(y_train_full[:val_start])
    X_val_sets.append(X_train_full[val_start: val_end])
    y_val_sets.append(y_train_full[val_start: val_end])

X_train_sets


(104758,) (104755,)
(209513,) (104755,)
(314268,) (104755,)
(419023,) (104755,)
(523778,) (104755,)


[<tf.Tensor: shape=(104758, 70), dtype=float64, numpy=
 array([[ 0.9132249 ,  0.9131219 , -1.58047303, ...,  0.87825308,
          1.00316397,  1.09089092],
        [ 0.82439921,  0.96512497, -1.58047303, ...,  0.73767417,
          0.87825188,  1.00316248],
        [ 0.73096249,  0.94779061, -1.58047303, ...,  0.62059541,
          0.73767302,  0.87825045],
        ...,
        [ 1.04049632,  0.79178142, -0.03461552, ...,  1.17141799,
          1.17704044,  1.17312218],
        [ 1.05281415,  0.61843787, -0.03461552, ...,  1.1511517 ,
          1.17141668,  1.17703887],
        [ 1.04567127,  0.722444  , -0.03461552, ...,  1.13299145,
          1.1511504 ,  1.17141511]])>,
 <tf.Tensor: shape=(209513, 70), dtype=float64, numpy=
 array([[ 0.9132249 ,  0.9131219 , -1.58047303, ...,  0.87825308,
          1.00316397,  1.09089092],
        [ 0.82439921,  0.96512497, -1.58047303, ...,  0.73767417,
          0.87825188,  1.00316248],
        [ 0.73096249,  0.94779061, -1.58047303, ...,  0.62

### Shallow multivariate MLP

In [5]:
# m1, m1_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=10_000, **get_MLP(X_train_full, model_prefix='Final'),
# )

# m1.evaluate(X_test, y_test)

# m1 = load_model(f'models/Final_MLP_shallow_multiv.keras')


### Deep multivariate MLP

In [6]:
# m2, m2_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=4000, **get_MLP(X_train_full, 10, model_prefix='Final'),
# )

# m2.evaluate(X_test, y_test)

# m2 = load_model(f'models/Final_MLP_deep_multiv.keras')


### Shallow multivariate BD-LSTM

In [7]:
# m3, m3_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=73, **get_LSTM(X_train_full, model_prefix='Final'),
# )

# m3.evaluate(X_test, y_test)
# m3.save_weights('models/Final_LSTM_shallow_multiv.weights.h5')

# m3 = load_lstm('Final_LSTM_shallow_multiv')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


### Deep multivariate BD-LSTM

In [8]:
# m4, m4_hist = train_model(
#     y_train_full, X_test, y_test, 
#     epochs=21, **get_LSTM(X_train_full, 2, model_prefix='Final'),
# )

# m4.evaluate(X_test, y_test)
# m4.save_weights('models/Final_LSTM_deep_multiv.weights.h5')

# m4 = load_lstm('Final_LSTM_deep_multiv')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


### Shallow univariate MLP


In [208]:
# m5, m5_hist = train_model(
#     y_train_full, X_test_uv, y_test, 
#     epochs=10_500, **get_MLP(X_train_uv, univariate=True, model_prefix='Final'),
# )

# m5.evaluate(X_test_uv, y_test)

# m5 = load_model(f'models/Final_MLP_shallow_univ.keras')


### Deep univariate MLP


In [10]:
# m6, m6_hist = train_model(
#     y_train_full, X_test_uv, y_test, 
#     epochs=3950, **get_MLP(X_train_uv, 10, univariate=True, model_prefix='Final'),
# )

# m6.evaluate(X_test_uv, y_test)

# m6 = load_model(f'models/Final_MLP_deep_univ.keras')


### Shallow univariate BD-LSTM


In [11]:
# m7, m7_hist = train_model(
#     y_train_full, X_test_uv, y_test, 
#     epochs=105, **get_LSTM(X_train_uv, univariate=True, model_prefix='Final'),
# )

# m7.evaluate(X_test_uv, y_test)
# m7.save_weights('models/Final_LSTM_shallow_univ.weights.h5')

# m7 = load_lstm('Final_LSTM_shallow_univ')


### Deep univariate BD-LSTM


In [12]:
# m8, m8_hist = train_model(
#     y_train_full, X_test_uv, y_test, 
#     epochs=30, **get_LSTM(X_train_uv, 2, univariate=True, model_prefix='Final'),
# )

# m8.evaluate(X_test_uv, y_test)

# m8.save_weights('models/Final_LSTM_deep_univ.weights.h5')

# m8 = load_lstm('Final_LSTM_deep_univ')



In [14]:
# m3, m3_history = train_model(y_train_full, X_test, y_test, X_train_full, m3, epochs=73) # Shallow multivariate LSTM
# m4, m4_history = train_model(y_train_full, X_test, y_test, X_train_full, m4, epochs=21) # Deep multivariate LSTM
# m7, m7_history = train_model(y_train_full, X_test_uv, y_test, X_train_uv, m7, epochs=105) # Shallow univariate LSTM
# m8, m8_history = train_model(y_train_full, X_test_uv, y_test, X_train_uv, m8, epochs=30) # Deep univariate LSTM

# model = extend_training(load_lstm('Final_LSTM_deep_univ'))


In [43]:
model = extend_training(load_lstm(get_latest_models('LSTM_deep_multiv'))) 

model.name


'Extended_Extended_Extended_Final_LSTM_deep_multiv'

In [33]:
# import hashlib

# proj = glob('models/*')
# proj_dct = dict()
# dwn = glob('/Users/mycomputer/Downloads/models*/*')
# dwn_dct = dict()

# for e in proj:
#     with open(e, 'rb') as f:
#        proj_dct[hashlib.md5(f.read()).hexdigest()] = e 

# for e in dwn:
#     with open(e, 'rb') as f:
#        dwn_dct[hashlib.md5(f.read()).hexdigest()] = e 

# print(sorted([v for k, v in dwn_dct.items() if k not in proj_dct and  '.csv' in v and 'train_times' not in v]))
# print([v for k, v in proj_dct.items() if k not in dwn_dct and '.csv' in v])


In [ ]:
hours = 8

In [ ]:
with tf.device('/CPU:0'):
    m4, m4_history = train_model(y_train_full, X_test, y_test, X_train_full, model, epochs=int(21/2*hours)) # Deep multivariate LSTM

# Get model results


In [65]:

model_files = get_latest_models()
models = dict()

for m, n, f in model_files.values:
    model = load_lstm(n) if 'LSTM' in m else load_model(f)
    models[m] = model
        
models_df = pd.DataFrame(models.items(), columns=['model', 'object'])

models_df['X_test'] = models_df.model.apply(lambda e: 'X_test_uv' if 'univ' in e else 'X_test')

models_df[['test_MAPE', 'test_MSE']] = models_df.apply(lambda d: eval(f"d['object'].evaluate({d['X_test']}, y_test)"), axis=1).apply(pd.Series)
models_df['test_RMSE'] = models_df.test_MSE**.5

models_df[['model', 'test_MAPE', 'test_RMSE']].to_csv('Results/model test results.csv', index=False)

models_df



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 153us/step - loss: 3.6530 - mean_squared_error: 93235.4219
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 4.3419 - mean_squared_error: 140531.1875
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 49s 10ms/step - loss: 4.8578 - mean_squared_error: 160166.5312
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 5.3894 - mean_squared_error: 191271.9219
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 193us/step - loss: 3.8033 - mean_squared_error: 106171.6641
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - loss: 5.1227 - mean_squared_error: 184611.8750
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 150us/step - loss: 4.2995 - mean_squared_error: 165347.5938
4911/4911 ━━━━━━━━━━━━━━━━━━━━ 1s 196us/step - loss: 4.1176 - mean_squared_error: 140319.0312


,model,object,X_test,test_MAPE,test_MSE,test_RMSE
0,MLP_shallow_multiv,"<Functional name=Final_MLP_shallow_multiv, bui...",X_test,4.592746,118508.984375,344.251339
1,LSTM_shallow_multiv,"<Functional name=Final_LSTM_shallow_multiv, bu...",X_test,5.072569,162968.937500,403.694114
2,LSTM_deep_multiv,"<Functional name=Final_LSTM_deep_multiv, built...",X_test,5.683590,186140.515625,431.440049
3,LSTM_shallow_univ,"<Functional name=Final_LSTM_shallow_univ, buil...",X_test_uv,6.017723,206086.812500,453.967854
4,MLP_deep_multiv,"<Functional name=Final_MLP_deep_multiv, built=...",X_test,4.703630,139177.281250,373.064715
5,LSTM_deep_univ,"<Functional name=Final_LSTM_deep_univ, built=T...",X_test_uv,5.837089,201973.609375,449.414741
6,MLP_shallow_univ,"<Functional name=Final_MLP_shallow_univ, built...",X_test_uv,4.716120,173877.687500,416.986436
7,MLP_deep_univ,"<Functional name=Final_MLP_deep_univ, built=True>",X_test_uv,4.641925,156586.593750,395.710240


In [69]:
# m4_daniels_y_pred = m4_daniels.predict(X_test)
# for i in range(2):
#     print(('H24' if i else 'H1'.rjust(3)) + ' MAPE:', MAPE(y_test[:,i], m4_y_pred[:,i]).numpy())
# for i in range(2):
#     print(('H24' if i else 'H1'.rjust(3)) + ' R2:', r2_score(y_test[:,i], m4_y_pred[:,i]))

# m4_daniels_history['epoch'] = (m4_daniels_history.index+1).to_list()
# mse_cols = [e for e in m4_daniels_history.columns if 'mean_squared_error' in e]
# m4_daniels_history[[e.replace('mean_squared_error', 'RMSE') for e in mse_cols]] = m4_daniels_history[mse_cols].apply(lambda d: d.apply(lambda e: e**.5), axis=1)

# fig, axes = plt.subplots(1, 2, figsize=(12,4), dpi=150)
# metrics = ['loss', 'RMSE']

# for i, ax in enumerate(axes):

#     metric = metrics[i]
#     label = metric.replace('loss', 'MAPE')

#     sns.lineplot(m4_daniels_history, x='epoch', y=metric, ax=ax, label='Train', alpha=0.5)
#     sns.lineplot(m4_daniels_history, x='epoch', y=f"val_{metric}", ax=ax, label='Test', alpha=0.5)

#     ax.set_ylim(0)
#     ax.set_xlabel('Epoch')
#     ax.set_ylabel(label)
#     # ax.set_yscale('symlog')

# plt.suptitle('M4')
# plt.savefig('LSTM_deep_mutliv continued fitting.png', dpi=300, facecolor='white', edgecolor='white', bbox_inches='tight')

# plt.show()
    


In [10]:
# param_grid={
#     'estimator__hidden_L1': [0.01, 0.1, 0.3],
#     'estimator__output_L1': [0.0, 0.01, 0.1, 0.3],
# }

# results = GridSearchCV(get_MLP, param_grid, epochs=100)

# with open('Results/MLP regularisation gridsearch results.json', 'w') as f:
#     f.write(dumps(results, indent=4))

# results



Searching hyperparameter space: 100%|████████████| 12/12 [59:49<00:00, 299.21s/it]


{'est_params': {'hidden_L1': 0.1, 'output_L1': 0.0},
 'opt_params': {},
 'params': {}}

In [78]:
# param_grid={
#     'batch_size': [2**8, 2**9, 2**10],
#     'optimizer__weight_decay': [1e-3, 1e-4, 1e-5],
#     'optimizer__beta_1': [.8, .9, .95],
# }

# results = GridSearchCV(get_MLP, param_grid, epochs=300)

# with open('Results/MLP gridsearch results.json', 'w') as f:
#     f.write(dumps(results, indent=4))

# results


{'opt_params': {'weight_decay': 1e-05, 'beta_1': 0.95},
 'params': {'batch_size': 256}}

In [81]:
# param_grid={
#     'batch_size': [2**8, 2**9, 2**10],
#     'optimizer__weight_decay': [1e-3, 1e-4, 1e-5],
#     'optimizer__beta_1': [.8, .9, .95],
# }

# results = GridSearchCV(get_LSTM, param_grid, epochs=10)

# with open('Results/BD-LSTM gridsearch results.json', 'w') as f:
#     f.write(dumps(results, indent=4))

# results


Searching hyperparameter space: 100%|██████████| 27/27 [54:11<00:00, 120.44s/it]


{'opt_params': {'weight_decay': 1e-05, 'beta_1': 0.8},
 'params': {'batch_size': 256}}

In [131]:
# from json import dumps, loads

# with open('.json', 'w') as f:
#     f.write(dumps(res))

# with open('weight decay comparison on shallow MLP.json', 'r') as f:
#     xx = loads(f.read())


In [88]:
# smry_df = pd.DataFrame(smry)
# smry_df = smry_df.melt(var_name='weight_decay', value_name='H1')
# smry_df.weight_decay.fillna(0, inplace=True)
# smry_df[['H1', 'H24']] = smry_df.H1.apply(pd.Series)

# smry_df

# smry_df.to_csv('.csv', index=False, lineterminator='\n')


In [60]:
# main_input = Input(shape=(X_train_sets[-1][0].shape[0],1), name="input")
# previous_layer = main_input
# # previous_layer = Bidirectional(LSTM(64, return_sequences=True), name=f"BD_1")(previous_layer)
# previous_layer = Bidirectional(LSTM(32), name=f"BD_2")(previous_layer)

# previous_layer = Dense(32, name=f"relu_{1}", activation='relu')(previous_layer)
# main_output = Dense(2, name=f"output")(previous_layer)

# def scheduler(epoch, lr):
#     return 0.001 if epoch % 2 == 1 else 0.01

# model_lstm = Model(inputs=main_input, outputs=main_output, name="BD-LSTM")
# model_lstm.compile(optimizer=Adam(
#     learning_rate=.001,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     weight_decay=None,
#     clipnorm=None,
#     clipvalue=None,
#     global_clipnorm=None,
#     use_ema=False,
#     ema_momentum=0.99,
#     ema_overwrite_frequency=None,
#     name='Adam',
# ), loss=MeanAbsolutePercentageError(), metrics=[MSE])
# lstm_history = model_lstm.fit(
#     X_train, y_train, verbose=1, validation_data=(X_val, y_val), shuffle=False,
#     epochs=1000, batch_size=2**5, 
#     callbacks=[
#         LearningRateScheduler(scheduler), 
#         EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True), 
#     ]
# )


444/444 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  LR=0.01  
 H1: 1.8942682  
H24: 3.261359  
  
 H1: 2.3578866  
H24: 3.7653885  

444/444 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  LR=0.001  
 H1: 2.2099416  
H24: 3.548185  

444/444 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  0.0001 if epoch % 2 == 1 else 0.001  (Epoch 154/1000)  
 H1: 1.4781239  
H24: 3.2230973  


In [198]:
# y_pred = model_lstm.predict(X_val)
# for i in range(2):
#     print(('H24' if i else 'H1'.rjust(3)) + ':', MAPE(y_val[:,i], y_pred[:,i]).numpy())

# lstm_history_df = pd.DataFrame(lstm_history.history)
# lstm_history_df['epoch'] = (lstm_history_df.index+1).to_list()
# mse_cols = [e for e in lstm_history_df.columns if 'mean_squared_error' in e]
# lstm_history_df[[e.replace('mean_squared_error', 'RMSE') for e in mse_cols]] = lstm_history_df[mse_cols].apply(lambda d: d.apply(lambda e: e**.5), axis=1)

# fig, axes = plt.subplots(1, 2, figsize=(18,6))
# metrics = ['loss', 'RMSE']

# for i, ax in enumerate(axes):

#     metric = metrics[i]
#     label = metric.replace('loss', 'MAPE')

#     sns.lineplot(lstm_history_df, x='epoch', y=metric, ax=ax, label='Test')
#     sns.lineplot(lstm_history_df, x='epoch', y=f"val_{metric}", ax=ax, label='Validation')

#     ax.set_ylim(0)
#     ax.set_xlabel('Epoch')
#     ax.set_ylabel(label)
    
# plt.show()
